<a href="https://colab.research.google.com/github/Chicane2002/Projects/blob/main/Sentiment_Analysis_GNEWS_SWIVEL_Kaggle_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import tf_keras
from tf_keras import Sequential
from tf_keras.layers import Dense, Dropout, LSTM, Embedding, LeakyReLU
from tf_keras import regularizers
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tf_keras.callbacks import EarlyStopping

training_data, validation_data, test_data = tfds.load(
    'imdb_reviews',
    split=['train[:80%]', 'train[80%:]', 'test'],
    as_supervised=True,
)

embed = hub.load("https://kaggle.com/models/google/gnews-swivel/frameworks/TensorFlow2/variations/tf2-preview-20dim/versions/1")

hub_layer = hub.KerasLayer(embed, input_shape=[], dtype=tf.string, trainable=True)

model = Sequential()
model.add(hub_layer)
model.add(Dense(64, activation=LeakyReLU(alpha=0.1), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(64, activation=LeakyReLU(alpha=0.1), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(32, activation=LeakyReLU(alpha=0.1), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(16, activation=LeakyReLU(alpha=0.1), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=4)

history = model.fit(training_data.shuffle(10000).batch(512),
                    validation_data=validation_data.batch(512),
                    epochs=25,
                    callbacks=[early_stopping], verbose=2
                    )
loss, accuracy = model.evaluate(test_data.batch(512),verbose=2)
print("Test Loss: ",loss)
print("Test Accuracy: ",accuracy)